In [1]:
import pandas as pd
import glob
import json
from tqdm import tqdm
import re
from datetime import datetime
import gensim
import logging
import os
import shutil

In [2]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [4]:
!pip install pysrt

    100% |████████████████████████████████| 112kB 3.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for pysrt ... done
  Stored in directory: /Users/npatta01/Library/Caches/pip/wheels/a6/95/51/25db5b533f7c8c3bccf661a7f2bf67caaf893f6f92bb37da33
Successfully built pysrt
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import pysrt


kaggle datasets download -d goweiting/ted-talks-transcript    
Use the Kaggle API to download dataset files.      
https://github.com/Kaggle/kaggle-api    

https://www.kaggle.com/goweiting/ted-talks-transcript/data

Doc2vec   
https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e

In [6]:
!ls -lah ~/.kaggle/datasets/goweiting/ted-talks-transcript/

ls: /Users/npatta01/.kaggle/datasets/goweiting/ted-talks-transcript/: No such file or directory


In [7]:
#!unzip /Users/nidhin/.kaggle/datasets/goweiting/ted-talks-transcript/tedDirector.zip

In [22]:
base_path= os.path.expanduser("~/.kaggle/datasets/goweiting/ted-talks-transcript")

In [9]:
#shutil.rmtree(base_path, ignore_errors=True)
#os.makedirs(base_path)

In [10]:
!ls /Users/nidhin.pattaniyil/.kaggle/datasets/goweiting/ted-talks-transcript


ls: /Users/nidhin.pattaniyil/.kaggle/datasets/goweiting/ted-talks-transcript: No such file or directory


In [11]:
!kaggle datasets download -d goweiting/ted-talks-transcript -p {base_path}/

100%|███████████████████████████████████████▉| 535M/535M [00:53<00:00, 7.96MB/s]
100%|████████████████████████████████████████| 535M/535M [00:53<00:00, 10.4MB/s]


In [12]:
!ls /Users/nidhin.pattaniyil/.kaggle/datasets/goweiting/ted-talks-transcript

ls: /Users/nidhin.pattaniyil/.kaggle/datasets/goweiting/ted-talks-transcript: No such file or directory


In [23]:
!ls {base_path}

subtitles                tedDirector.stats.csv    ted_metadata_kaggle.csv
ted-talks-transcript.zip tedDirector.zip          ted_metadata_youtube.csv


In [25]:
!unzip -q -o {base_path}/ted-talks-transcript.zip -d {base_path}
!unzip -q -o {base_path}/tedDirector.zip -d {base_path}

In [26]:
print(f"{base_path}subtitles/*.info.json")

/Users/npatta01/.kaggle/datasets/goweiting/ted-talks-transcriptsubtitles/*.info.json


In [29]:
meta_data_files = glob.glob(f"{base_path}/subtitles/*/*.info.json")




In [30]:
len(meta_data_files)

2531

In [31]:
def parse_video_info(metadata):
    keys = ["webpage_url", "view_count", "upload_date", "title", "thumbnail" 
            , "duration" , "dislike_count" , "like_count"
            , "description", "categories" ,"id", "average_rating"
            ]
    
    new_dict = {}
    for key in keys:
        new_dict[key] = metadata.get(key,None)
        
    return new_dict

def get_metadata(meta_data_files) -> pd.DataFrame:
    parsed_metadatas = []
    for file in tqdm(meta_data_files):
        with open(file) as f:
            content = json.load(f)
        
        parsed_metadata = parse_video_info(content)
        parsed_metadatas.append(parsed_metadata)

    return pd.DataFrame(parsed_metadatas)


In [32]:
metadata_df = get_metadata(meta_data_files)

100%|██████████| 2531/2531 [00:01<00:00, 1624.45it/s]


In [33]:
metadata_df.head()

,average_rating,categories,description,dislike_count,duration,id,like_count,thumbnail,title,upload_date,view_count,webpage_url
0,4.729469,[Nonprofits & Activism],http://www.ted.com When German psychologist In...,14.0,830,Jc2F3-nawnI,193.0,https://i.ytimg.com/vi/Jc2F3-nawnI/hqdefault.jpg,Inge Missmahl brings peace to the minds of Afg...,20100929,20695,https://www.youtube.com/watch?v=Jc2F3-nawnI
1,4.940791,[News & Politics],"As a child growing up in North Korea, Hyeonseo...",2151.0,735,PdxPCeWw75k,143165.0,https://i.ytimg.com/vi/PdxPCeWw75k/maxresdefau...,My escape from North Korea | Hyeonseo Lee,20130320,6702386,https://www.youtube.com/watch?v=PdxPCeWw75k
2,4.554828,[News & Politics],"http://www.ted.com Every day, in a city the si...",68.0,938,CLWRclarri0,543.0,https://i.ytimg.com/vi/CLWRclarri0/hqdefault.jpg,Carolyn Steel: How food shapes our cities,20091005,125975,https://www.youtube.com/watch?v=CLWRclarri0
3,4.706122,[Science & Technology],http://www.ted.com With his team at SENSEable ...,54.0,947,CijsvAGU6-c,681.0,https://i.ytimg.com/vi/CijsvAGU6-c/hqdefault.jpg,Carlo Ratti: Architecture that senses and resp...,20110503,78537,https://www.youtube.com/watch?v=CijsvAGU6-c
4,4.732777,[Travel & Events],"Almost everything we own and use, at some poin...",96.0,684,j7RsRnYlz7I,1341.0,https://i.ytimg.com/vi/j7RsRnYlz7I/maxresdefau...,Rose George: Inside the secret shipping industry,20131213,144699,https://www.youtube.com/watch?v=j7RsRnYlz7I


In [36]:
subtitle_files = glob.glob(f"{base_path}/subtitles/*/*.en.vtt")


In [37]:
subtitle_files[0:2]

['/Users/npatta01/.kaggle/datasets/goweiting/ted-talks-transcript/subtitles/Jc2F3-nawnI/Jc2F3-nawnI.en.vtt',
 '/Users/npatta01/.kaggle/datasets/goweiting/ted-talks-transcript/subtitles/PdxPCeWw75k/PdxPCeWw75k.en.vtt']

In [38]:
def parse_subtitle(file_name) -> str:
    subs = pysrt.open(file_name)
    content = ""
    for s in subs:
        content = content + s.text_without_tags + "\n"
    return content

def get_subtitles(subtitle_files) -> pd.DataFrame :
    parsed_subtitles = []
    for file in tqdm(subtitle_files):
        
        res = re.match('.*/(.*).en.vtt',file)
        talk_id  = res.group(1)
        content = parse_subtitle(file)
        
        parsed_subtitles.append ({'id': talk_id , 'transcript': content})
        
    return pd.DataFrame(parsed_subtitles)
        

In [39]:
subtitle_files_df = get_subtitles(subtitle_files)

100%|██████████| 2490/2490 [00:20<00:00, 122.50it/s]


In [40]:
subtitle_files_df.head()

,id,transcript
0,Jc2F3-nawnI,So I want to tell you a story -- an encouragin...
1,PdxPCeWw75k,Translator: Joseph Geni\nReviewer: Morton Bast...
2,CLWRclarri0,How do you feed a city?\nIt's one of the great...
3,CijsvAGU6-c,"Good afternoon, everybody.\nI've got something..."
4,j7RsRnYlz7I,"A couple of years ago,\nHarvard Business Schoo..."


In [41]:
len(metadata_df)

2531

In [42]:
len(subtitle_files_df)

2490

In [43]:
df = pd.merge(metadata_df,subtitle_files_df, on='id')

In [44]:
datetime.strptime('20100929', '%Y%m%d')


datetime.datetime(2010, 9, 29, 0, 0)

In [45]:
df.head()

,average_rating,categories,description,dislike_count,duration,id,like_count,thumbnail,title,upload_date,view_count,webpage_url,transcript
0,4.729469,[Nonprofits & Activism],http://www.ted.com When German psychologist In...,14.0,830,Jc2F3-nawnI,193.0,https://i.ytimg.com/vi/Jc2F3-nawnI/hqdefault.jpg,Inge Missmahl brings peace to the minds of Afg...,20100929,20695,https://www.youtube.com/watch?v=Jc2F3-nawnI,So I want to tell you a story -- an encouragin...
1,4.940791,[News & Politics],"As a child growing up in North Korea, Hyeonseo...",2151.0,735,PdxPCeWw75k,143165.0,https://i.ytimg.com/vi/PdxPCeWw75k/maxresdefau...,My escape from North Korea | Hyeonseo Lee,20130320,6702386,https://www.youtube.com/watch?v=PdxPCeWw75k,Translator: Joseph Geni\nReviewer: Morton Bast...
2,4.554828,[News & Politics],"http://www.ted.com Every day, in a city the si...",68.0,938,CLWRclarri0,543.0,https://i.ytimg.com/vi/CLWRclarri0/hqdefault.jpg,Carolyn Steel: How food shapes our cities,20091005,125975,https://www.youtube.com/watch?v=CLWRclarri0,How do you feed a city?\nIt's one of the great...
3,4.706122,[Science & Technology],http://www.ted.com With his team at SENSEable ...,54.0,947,CijsvAGU6-c,681.0,https://i.ytimg.com/vi/CijsvAGU6-c/hqdefault.jpg,Carlo Ratti: Architecture that senses and resp...,20110503,78537,https://www.youtube.com/watch?v=CijsvAGU6-c,"Good afternoon, everybody.\nI've got something..."
4,4.732777,[Travel & Events],"Almost everything we own and use, at some poin...",96.0,684,j7RsRnYlz7I,1341.0,https://i.ytimg.com/vi/j7RsRnYlz7I/maxresdefau...,Rose George: Inside the secret shipping industry,20131213,144699,https://www.youtube.com/watch?v=j7RsRnYlz7I,"A couple of years ago,\nHarvard Business Schoo..."


In [46]:
?df.to_csv

In [47]:
df.to_csv("talks.csv.gzip",compression='gzip')